# Part 3 — Statistical tests & comparison
Paired statistical tests between methods.

We use paired t-test on ARI / NMI / Silhouette across repeated seeds.
If you only have one seed, expand by running multiple seeds and re-saving results.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
os.chdir("..")
sys.path.append(os.getcwd())

from src.utils.random import set_seed
from src.utils.preprocessing import StandardScaler


In [6]:
import os
os.chdir('..')
import sys
sys.path.append('.')


In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv("results/all_results.csv")
df.head()


,silhouette,davies_bouldin,calinski_harabasz,wcss,ari,nmi,purity,confusion_matrix,confusion_labels,experiment,...,init,k,time_sec,iters,log_likelihood,bic,aic,cov,dim,rec_mse
0,0.344734,1.309289,267.680436,11595.683313,0.676505,0.562021,0.912127,[[344 13]\n [ 37 175]],[0 1],E1,...,kmeans++,2,0.001243,8,NaN,NaN,NaN,NaN,NaN,NaN
1,0.343382,1.320510,267.691716,11595.526607,0.653625,0.532408,0.905097,[[339 18]\n [ 36 176]],[0 1],E1,...,random,2,0.000746,6,NaN,NaN,NaN,NaN,NaN,NaN
2,0.147455,1.966664,67.239679,13792.868977,0.321036,0.279951,0.789104,[[ 0 94 263]\n [ 15 171 26]\n [ 0 0 0]],[0 1 2],E2,...,NaN,3,0.075895,31,428.803587,8575.743032,2116.392827,full,NaN,NaN
3,0.094505,2.005950,32.547104,11652.206140,0.146592,0.182262,0.753954,[[ 2 311 4 0 4 1 0 3 32]\n [ 0 ...,[0 1 2 3 4 5 6 7 8],E2,...,NaN,9,0.286510,45,-2594.670231,9902.843625,6675.340463,tied,NaN,NaN
4,0.048151,2.172302,75.472364,7706.133782,0.150672,0.338435,0.926186,[[64 52 26 78 5 0 52 0 58 22]\n [ 1 1 32 ...,[0 1 2 3 4 5 6 7 8 9],E2,...,NaN,10,0.012923,16,-12339.555436,28542.534056,25897.110872,diag,NaN,NaN


In [8]:
# Example: compare PCA-KMeans vs AE-KMeans across the same dim values using ARI
# (Assumes you ran DIM_LIST and k=2)
pca = df[(df.experiment=="E3") & (df.method=="KMeans")].copy()
ae  = df[(df.experiment=="E5") & (df.method=="KMeans")].copy()

merged = pd.merge(pca, ae, on="dim", suffixes=("_pca","_ae"))
print("rows:", len(merged))
merged[["dim","ari_pca","ari_ae","silhouette_pca","silhouette_ae"]]


rows: 5


,dim,ari_pca,ari_ae,silhouette_pca,silhouette_ae
0,2.0,0.659231,0.245562,0.508469,0.682248
1,5.0,0.670721,0.391333,0.393223,0.604034
2,10.0,0.676505,0.004828,0.357745,0.859981
3,15.0,0.676505,0.296726,0.348078,0.686416
4,20.0,0.676505,0.415795,0.345859,0.664024


In [9]:
# Paired t-test implemented from scratch (normal approximation)
def paired_t_test(x, y):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    d = x - y
    n = len(d)
    md = d.mean()
    sd = d.std(ddof=1)
    t = md / (sd / np.sqrt(n) + 1e-12)
    # p-value would require CDF; report t and direction.
    return float(t), float(md), float(sd), int(n)

t_ari, md, sd, n = paired_t_test(merged["ari_pca"], merged["ari_ae"])
print("Paired t-test (ARI)  t=", t_ari, "mean_diff=", md, "std_diff=", sd, "n=", n)


Paired t-test (ARI)  t= 5.448763481507266 mean_diff= 0.401044288938853 std_diff= 0.16458088061386938 n= 5
